In [ ]:
import _pickle as pickle
import torch
import matplotlib.pyplot as plt
import numpy as np
import math
from tqdm import tqdm
import re

In [ ]:
src = []
with open("./fairseq/raw_dataset/cnndm/raw/test.src","r") as f:
    for line in f:
        src.append(line)

In [ ]:
def extract(idx):
    attention = pickle.load(open("./fairseq/result/summarization_vanilla_cnndm/attention/attention_distribution_"+str(idx)+".dat","rb"))
    avg_attention = torch.sum(attention, dim=1).cpu().numpy()
    tokens = src[idx].split(" ")[:-1]
    sentences = src[idx].split(" . ")[:-1]      
    attention = 0.0
    attn_sentence = []
    l = min(len(tokens),len(avg_attention))
    for i in range(l):
        if tokens[i] == '.':
            attn_sentence.append(attention)
            attention = 0.0
        else:
            attention += avg_attention[i]  
    attn_sorted = sorted(zip(attn_sentence, sentences), key=lambda pair: pair[0])
    attn, chosen = zip(*attn_sorted[-2:])
    result = " ".join(chosen)
    with open("./fairseq/result/summarization_extractive_cnndm-nounk/models/model."+str(idx)+".txt","w") as f:
        f.write(result)    

In [ ]:
for i in tqdm(range(11490)):
    extract(i)